# Investment Strategy Simulator

This interactive notebook allows you to test different asset allocation strategies across various market conditions. You can visualize how portfolio composition affects risk-return profiles with interactive Efficient Frontier plots.

## What is Portfolio Optimization?

Portfolio optimization is the process of selecting the best asset allocation that provides the highest expected return for a given level of risk, or the lowest risk for a given level of expected return.

## What You'll Learn

- How to build diversified portfolios with different asset classes
- Calculating expected returns, volatility, and correlations
- Creating the Efficient Frontier for optimal portfolios
- Visualizing risk-return tradeoffs
- Testing portfolio performance across different market conditions
- Implementing portfolio rebalancing strategies


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import datetime as dt
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import plotly.graph_objects as go
import plotly.express as px
from scipy.optimize import minimize

# Set styling
plt.style.use("ggplot")
sns.set_palette("viridis")

# Suppress warnings
import warnings

warnings.filterwarnings("ignore")

## Loading Historical Asset Data

We'll start by defining a set of assets across different classes (stocks, bonds, commodities, etc.) and downloading their historical data.

These assets will be used to build and test different portfolio strategies.


In [2]:
# Define asset classes and tickers
assets = {
    "US Large Cap Stocks": "SPY",  # S&P 500 ETF
    "US Small Cap Stocks": "IWM",  # Russell 2000 ETF
    "International Stocks": "EFA",  # MSCI EAFE ETF
    "Emerging Markets": "EEM",  # MSCI Emerging Markets ETF
    "US Bonds": "AGG",  # US Aggregate Bond ETF
    "Treasury Bonds": "TLT",  # 20+ Year Treasury ETF
    "Corporate Bonds": "LQD",  # Investment Grade Corporate Bond ETF
    "Real Estate": "VNQ",  # Vanguard Real Estate ETF
    "Gold": "GLD",  # Gold ETF
    "Commodities": "DBC",  # Commodity Index ETF
}


# Function to download historical data
def get_asset_data(tickers, start_date, end_date):
    """Download historical price data for a list of tickers"""
    try:
        data = yf.download(list(tickers.values()), start=start_date, end=end_date)["Adj Close"]
        # Rename columns from tickers to asset names
        ticker_to_name = {v: k for k, v in tickers.items()}
        data.columns = [ticker_to_name.get(col, col) for col in data.columns]
        return data
    except Exception as e:
        print(f"Error downloading data: {e}")
        return None


# Set default date range (5 years)
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=5 * 365)

# Download data
print("Downloading historical asset data...")
asset_prices = get_asset_data(assets, start_date, end_date)

if asset_prices is not None:
    # Calculate daily returns
    returns = asset_prices.pct_change().dropna()

    # Display basic statistics
    print(f"Data period: {returns.index[0].date()} to {returns.index[-1].date()}")
    print(f"Number of trading days: {len(returns)}")

    # Show the first few rows of data
    display(asset_prices.head())

    # Plot price history (normalized to 100)
    plt.figure(figsize=(12, 6))
    (asset_prices / asset_prices.iloc[0] * 100).plot()
    plt.title("Historical Asset Prices (Normalized to 100)")
    plt.ylabel("Normalized Price")
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.show()
else:
    print("Failed to download asset data. Please check your internet connection.")

Failed to get ticker 'GLD' reason: HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000017034ECF800>: Failed to resolve 'fc.yahoo.com' ([Errno 11004] getaddrinfo failed)"))
Failed to get ticker 'IWM' reason: HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000017034F2E2D0>: Failed to resolve 'fc.yahoo.com' ([Errno 11004] getaddrinfo failed)"))
Failed to get ticker 'DBC' reason: HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000017034F2E690>: Failed to resolve 'fc.yahoo.com' ([Errno 11004] getaddrinfo failed)"))
Failed to get ticker 'EFA' reason: HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by

YF.download() has changed argument auto_adjust default to True


IndexError: index 0 is out of bounds for axis 0 with size 0

## Portfolio Analysis Functions

Let's create functions to analyze portfolio performance, calculate risk-return metrics, and generate the Efficient Frontier.


In [ ]:
def calculate_portfolio_performance(weights, returns):
    """Calculate expected return and risk of portfolio"""
    # Convert weights to numpy array if needed
    weights = np.array(weights)

    # Calculate expected returns (annualized)
    expected_returns = np.sum(returns.mean() * weights) * 252

    # Calculate expected volatility (annualized)
    expected_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))

    # Calculate Sharpe Ratio (assuming risk-free rate of 0% for simplicity)
    sharpe_ratio = expected_returns / expected_volatility

    return expected_returns, expected_volatility, sharpe_ratio


def generate_random_portfolios(returns, num_portfolios=10000):
    """Generate random portfolio allocations"""
    results = []
    asset_count = len(returns.columns)

    for _ in range(num_portfolios):
        # Generate random weights
        weights = np.random.random(asset_count)
        weights = weights / np.sum(weights)  # Normalize to sum to 1

        # Calculate portfolio performance
        expected_return, volatility, sharpe = calculate_portfolio_performance(weights, returns)

        # Store results
        results.append(
            {
                "Return": expected_return,
                "Volatility": volatility,
                "Sharpe": sharpe,
                "Weights": weights,
            }
        )

    return pd.DataFrame(results)


def optimize_portfolio(returns, target_return=None, target="sharpe"):
    """Find the optimal portfolio based on different objectives"""
    asset_count = len(returns.columns)
    args = (returns,)
    constraints = {"type": "eq", "fun": lambda x: np.sum(x) - 1}  # Weights sum to 1
    bounds = tuple((0, 1) for _ in range(asset_count))  # Weights between 0 and 1

    if target == "sharpe":
        # Maximize Sharpe ratio (minimize negative Sharpe)
        def objective(weights, returns):
            return -calculate_portfolio_performance(weights, returns)[2]

    elif target == "return":
        # Maximize return
        def objective(weights, returns):
            return -calculate_portfolio_performance(weights, returns)[0]

    elif target == "volatility":
        # Minimize volatility
        def objective(weights, returns):
            return calculate_portfolio_performance(weights, returns)[1]

    elif target == "target_return":
        if target_return is None:
            raise ValueError("Target return must be specified")

        # Minimize volatility subject to target return
        def objective(weights, returns):
            return calculate_portfolio_performance(weights, returns)[1]

        constraints = (
            {"type": "eq", "fun": lambda x: np.sum(x) - 1},  # Weights sum to 1
            {
                "type": "eq",
                "fun": lambda x: calculate_portfolio_performance(x, returns)[0] - target_return,
            },  # Target return
        )

    # Initial guess: equal weights
    initial_weights = np.array([1 / asset_count] * asset_count)

    # Run optimization
    result = minimize(
        objective,
        initial_weights,
        args=args,
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
    )

    # Get optimal weights
    optimal_weights = result["x"]

    # Calculate performance with optimal weights
    expected_return, volatility, sharpe = calculate_portfolio_performance(optimal_weights, returns)

    return {
        "Return": expected_return,
        "Volatility": volatility,
        "Sharpe": sharpe,
        "Weights": optimal_weights,
    }


def generate_efficient_frontier(returns, points=100):
    """Generate the efficient frontier"""
    # Find minimum volatility portfolio
    min_vol_port = optimize_portfolio(returns, target="volatility")
    min_ret = min_vol_port["Return"]

    # Find maximum return portfolio
    max_return_port = optimize_portfolio(returns, target="return")
    max_ret = max_return_port["Return"]

    # Create range of returns
    target_returns = np.linspace(min_ret, max_ret, points)
    efficient_portfolios = []

    # For each return target, optimize for minimum volatility
    for target in target_returns:
        try:
            port = optimize_portfolio(returns, target_return=target, target="target_return")
            efficient_portfolios.append(port)
        except:
            # Skip if optimization fails
            continue

    return pd.DataFrame(efficient_portfolios)


def plot_efficient_frontier(returns, random_portfolios=None, show_assets=True, num_random=10000):
    """Plot the efficient frontier with optional random portfolios"""
    # Generate efficient frontier
    efficient_frontier = generate_efficient_frontier(returns)

    # Get max Sharpe ratio portfolio
    max_sharpe_port = optimize_portfolio(returns, target="sharpe")

    # Get min volatility portfolio
    min_vol_port = optimize_portfolio(returns, target="volatility")

    # Create plot
    plt.figure(figsize=(12, 8))

    # Plot random portfolios if provided or requested
    if random_portfolios is None and num_random > 0:
        random_portfolios = generate_random_portfolios(returns, num_portfolios=num_random)

    if random_portfolios is not None:
        plt.scatter(
            random_portfolios["Volatility"],
            random_portfolios["Return"],
            alpha=0.1,
            color="lightblue",
            label="Random Portfolios",
        )

    # Plot efficient frontier
    plt.plot(
        efficient_frontier["Volatility"],
        efficient_frontier["Return"],
        "b-",
        label="Efficient Frontier",
    )

    # Plot max Sharpe portfolio
    plt.scatter(
        max_sharpe_port["Volatility"],
        max_sharpe_port["Return"],
        marker="*",
        color="green",
        s=200,
        label="Maximum Sharpe Ratio",
    )

    # Plot min volatility portfolio
    plt.scatter(
        min_vol_port["Volatility"],
        min_vol_port["Return"],
        marker="o",
        color="red",
        s=200,
        label="Minimum Volatility",
    )

    # Plot individual assets if requested
    if show_assets:
        # Calculate annualized returns and volatilities for individual assets
        asset_returns = returns.mean() * 252
        asset_volatility = returns.std() * np.sqrt(252)

        # Plot each asset
        for i, asset in enumerate(returns.columns):
            plt.scatter(asset_volatility[i], asset_returns[i], marker="o", s=100, label=asset)

    # Set labels and title
    plt.xlabel("Expected Volatility (Standard Deviation)")
    plt.ylabel("Expected Annual Return")
    plt.title("Efficient Frontier of Optimal Portfolios")
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.grid(True)
    plt.tight_layout()

    return plt.gcf()  # Return the figure


def display_portfolio(weights, returns, title="Portfolio Analysis"):
    """Display portfolio weights and performance metrics"""
    # Calculate performance
    expected_return, volatility, sharpe = calculate_portfolio_performance(weights, returns)

    # Create DataFrame for weights
    asset_names = returns.columns
    weights_df = pd.DataFrame({"Asset": asset_names, "Weight": weights * 100})
    weights_df = weights_df.sort_values("Weight", ascending=False)

    # Display metrics
    display(HTML(f"<h3>{title}</h3>"))
    display(HTML(f"<p><b>Expected Annual Return:</b> {expected_return:.2%}</p>"))
    display(HTML(f"<p><b>Expected Volatility:</b> {volatility:.2%}</p>"))
    display(HTML(f"<p><b>Sharpe Ratio:</b> {sharpe:.2f}</p>"))

    # Plot weights
    plt.figure(figsize=(10, 6))

    # Bar chart of weights
    plt.subplot(1, 2, 1)
    sns.barplot(x="Weight", y="Asset", data=weights_df)
    plt.title("Asset Allocation")
    plt.xlabel("Weight (%)")

    # Pie chart of weights
    plt.subplot(1, 2, 2)
    plt.pie(weights_df["Weight"], labels=weights_df["Asset"], autopct="%1.1f%%")
    plt.title("Portfolio Composition")
    plt.axis("equal")

    plt.tight_layout()
    plt.show()

    return weights_df

## Asset Correlation Analysis

Understanding correlations between assets is crucial for building diversified portfolios. Let's visualize these relationships.


In [ ]:
if "returns" in globals():
    # Calculate correlation matrix
    correlation_matrix = returns.corr()

    # Plot correlation heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(
        correlation_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1, linewidths=0.5, fmt=".2f"
    )
    plt.title("Asset Correlation Matrix")
    plt.tight_layout()
    plt.show()

    # Calculate and display annualized return and risk for each asset
    annual_returns = returns.mean() * 252
    annual_volatility = returns.std() * np.sqrt(252)
    annual_sharpe = annual_returns / annual_volatility

    asset_metrics = pd.DataFrame(
        {
            "Annual Return": annual_returns,
            "Annual Volatility": annual_volatility,
            "Sharpe Ratio": annual_sharpe,
        }
    )

    # Sort by Sharpe ratio
    asset_metrics = asset_metrics.sort_values("Sharpe Ratio", ascending=False)

    # Format as percentages
    asset_metrics["Annual Return"] = asset_metrics["Annual Return"].apply(lambda x: f"{x:.2%}")
    asset_metrics["Annual Volatility"] = asset_metrics["Annual Volatility"].apply(
        lambda x: f"{x:.2%}"
    )
    asset_metrics["Sharpe Ratio"] = asset_metrics["Sharpe Ratio"].apply(lambda x: f"{x:.2f}")

    display(HTML("<h3>Asset Performance Metrics</h3>"))
    display(asset_metrics)

    # Plot risk-return scatter plot
    plt.figure(figsize=(10, 8))
    ax = plt.scatter(returns.std() * np.sqrt(252), returns.mean() * 252, s=100)

    # Add labels to each point
    for i, asset in enumerate(returns.columns):
        plt.annotate(
            asset,
            (returns.std()[i] * np.sqrt(252), returns.mean()[i] * 252),
            xytext=(5, 5),
            textcoords="offset points",
        )

    plt.xlabel("Annual Volatility (Standard Deviation)")
    plt.ylabel("Annual Expected Return")
    plt.title("Risk-Return Profile of Individual Assets")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

## Efficient Frontier Visualization

The efficient frontier represents the set of optimal portfolios that offer the highest expected return for a defined level of risk.


In [ ]:
if "returns" in globals():
    # Plot efficient frontier with random portfolios
    _ = plot_efficient_frontier(returns, num_random=5000, show_assets=True)

    # Display optimal portfolios
    max_sharpe_port = optimize_portfolio(returns, target="sharpe")
    min_vol_port = optimize_portfolio(returns, target="volatility")

    # Display max Sharpe portfolio
    max_sharpe_weights = max_sharpe_port["Weights"]
    display_portfolio(max_sharpe_weights, returns, "Maximum Sharpe Ratio Portfolio")

    # Display min volatility portfolio
    min_vol_weights = min_vol_port["Weights"]
    display_portfolio(min_vol_weights, returns, "Minimum Volatility Portfolio")

## Interactive Portfolio Builder

Create and test your own portfolio allocation interactively!


In [ ]:
if "returns" in globals():
    # Create sliders for each asset
    sliders = {}
    for asset in returns.columns:
        sliders[asset] = widgets.FloatSlider(
            value=10.0,  # Default 10% allocation
            min=0.0,
            max=100.0,
            step=1.0,
            description=asset,
            disabled=False,
            continuous_update=False,
            orientation="horizontal",
            readout=True,
            readout_format=".0f",
            layout=widgets.Layout(width="80%"),
        )

    # Create a reset button and analyze button
    reset_button = widgets.Button(
        description="Reset to Equal",
        disabled=False,
        button_style="warning",
        tooltip="Reset to equal allocation",
        icon="refresh",
    )

    analyze_button = widgets.Button(
        description="Analyze Portfolio",
        disabled=False,
        button_style="success",
        tooltip="Analyze the current allocation",
        icon="check",
    )

    # Create output area
    output_area = widgets.Output()

    # Add a total label
    total_label = widgets.Label(value="Total allocation: 100.0%")

    # Function to update total
    def update_total(change):
        total = sum(slider.value for slider in sliders.values())
        total_label.value = f"Total allocation: {total:.1f}%"
        # Change color based on validity
        if abs(total - 100.0) < 0.1:  # Allow for small floating point errors
            total_label.style = {"color": "green"}
        else:
            total_label.style = {"color": "red"}

    # Connect update function to each slider
    for slider in sliders.values():
        slider.observe(update_total, names="value")

    # Reset function
    def reset_to_equal(b):
        equal_weight = 100.0 / len(sliders)
        for slider in sliders.values():
            slider.value = equal_weight

    reset_button.on_click(reset_to_equal)

    # Analyze function
    def analyze_portfolio(b):
        with output_area:
            clear_output()

            # Get weights
            weights = np.array([slider.value / 100.0 for slider in sliders.values()])

            # Check if weights sum to approximately 1
            if not np.isclose(sum(weights), 1.0, atol=0.01):
                print(f"Warning: Weights sum to {sum(weights):.2f}, not 1.0. Normalizing...")
                weights = weights / sum(weights)

            # Display portfolio analysis
            weights_df = display_portfolio(weights, returns, "Custom Portfolio Analysis")

            # Plot efficient frontier with this portfolio
            fig = plot_efficient_frontier(returns, num_random=1000, show_assets=False)

            # Calculate portfolio performance
            expected_return, volatility, sharpe = calculate_portfolio_performance(weights, returns)

            # Add custom portfolio to plot
            plt.scatter(
                volatility,
                expected_return,
                marker="X",
                color="purple",
                s=200,
                label="Your Portfolio",
            )

            plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
            plt.show()

            # Comparison to optimal portfolios
            max_sharpe_port = optimize_portfolio(returns, target="sharpe")
            min_vol_port = optimize_portfolio(returns, target="volatility")

            comparison = pd.DataFrame(
                [
                    {
                        "Portfolio": "Your Portfolio",
                        "Expected Return": f"{expected_return:.2%}",
                        "Volatility": f"{volatility:.2%}",
                        "Sharpe Ratio": f"{sharpe:.2f}",
                    },
                    {
                        "Portfolio": "Max Sharpe Portfolio",
                        "Expected Return": f"{max_sharpe_port['Return']:.2%}",
                        "Volatility": f"{max_sharpe_port['Volatility']:.2%}",
                        "Sharpe Ratio": f"{max_sharpe_port['Sharpe']:.2f}",
                    },
                    {
                        "Portfolio": "Min Volatility Portfolio",
                        "Expected Return": f"{min_vol_port['Return']:.2%}",
                        "Volatility": f"{min_vol_port['Volatility']:.2%}",
                        "Sharpe Ratio": f"{min_vol_port['Sharpe']:.2f}",
                    },
                ]
            )

            display(HTML("<h3>Portfolio Comparison</h3>"))
            display(comparison)

    analyze_button.on_click(analyze_portfolio)

    # Display widgets
    display(HTML("<h2>Interactive Portfolio Builder</h2>"))
    display(
        HTML("<p>Adjust the sliders to allocate your portfolio. The total should sum to 100%.</p>")
    )

    for slider in sliders.values():
        display(slider)

    display(total_label)
    display(widgets.HBox([reset_button, analyze_button]))
    display(output_area)

    # Initialize with equal weights
    reset_to_equal(None)

## Market Scenario Analysis

Test how different portfolios perform under various market conditions (bull market, bear market, etc.).


In [ ]:
if "returns" in globals():
    # Define market scenarios
    scenarios = {
        "Bull Market": {
            "description": "Strong economic growth with rising stock markets",
            "multipliers": {
                "US Large Cap Stocks": 1.5,
                "US Small Cap Stocks": 1.8,
                "International Stocks": 1.3,
                "Emerging Markets": 1.7,
                "US Bonds": 0.5,
                "Treasury Bonds": 0.3,
                "Corporate Bonds": 0.7,
                "Real Estate": 1.2,
                "Gold": 0.6,
                "Commodities": 1.1,
            },
        },
        "Bear Market": {
            "description": "Economic downturn with declining stock markets",
            "multipliers": {
                "US Large Cap Stocks": -1.2,
                "US Small Cap Stocks": -1.5,
                "International Stocks": -1.3,
                "Emerging Markets": -1.6,
                "US Bonds": 1.2,
                "Treasury Bonds": 1.5,
                "Corporate Bonds": 0.8,
                "Real Estate": -0.8,
                "Gold": 1.3,
                "Commodities": -0.7,
            },
        },
        "High Inflation": {
            "description": "Rising prices with central banks raising interest rates",
            "multipliers": {
                "US Large Cap Stocks": 0.7,
                "US Small Cap Stocks": 0.8,
                "International Stocks": 0.6,
                "Emerging Markets": 0.5,
                "US Bonds": -0.8,
                "Treasury Bonds": -1.2,
                "Corporate Bonds": -0.7,
                "Real Estate": 0.9,
                "Gold": 1.5,
                "Commodities": 1.8,
            },
        },
        "Recession": {
            "description": "Economic contraction with high unemployment",
            "multipliers": {
                "US Large Cap Stocks": -1.0,
                "US Small Cap Stocks": -1.4,
                "International Stocks": -1.1,
                "Emerging Markets": -1.2,
                "US Bonds": 1.0,
                "Treasury Bonds": 1.4,
                "Corporate Bonds": 0.5,
                "Real Estate": -1.0,
                "Gold": 1.2,
                "Commodities": -0.8,
            },
        },
        "Tech Boom": {
            "description": "Strong growth in technology sector",
            "multipliers": {
                "US Large Cap Stocks": 1.8,
                "US Small Cap Stocks": 1.6,
                "International Stocks": 1.2,
                "Emerging Markets": 1.5,
                "US Bonds": 0.4,
                "Treasury Bonds": 0.2,
                "Corporate Bonds": 0.6,
                "Real Estate": 0.8,
                "Gold": 0.5,
                "Commodities": 0.7,
            },
        },
    }

    # Create portfolio options
    portfolios = {
        "Conservative": np.array([0.15, 0.05, 0.05, 0.05, 0.25, 0.20, 0.15, 0.05, 0.03, 0.02]),
        "Balanced": np.array([0.25, 0.10, 0.10, 0.05, 0.15, 0.10, 0.10, 0.05, 0.05, 0.05]),
        "Aggressive": np.array([0.35, 0.15, 0.15, 0.10, 0.05, 0.05, 0.05, 0.05, 0.03, 0.02]),
        "Max Sharpe": optimize_portfolio(returns, target="sharpe")["Weights"],
        "Min Volatility": optimize_portfolio(returns, target="volatility")["Weights"],
    }

    # Create scenario dropdown
    scenario_dropdown = widgets.Dropdown(
        options=list(scenarios.keys()),
        value=list(scenarios.keys())[0],
        description="Scenario:",
        style={"description_width": "initial"},
    )

    # Create output area
    scenario_output = widgets.Output()

    # Function to analyze scenarios
    def analyze_scenario(scenario_name):
        scenario = scenarios[scenario_name]

        # Create modified returns based on scenario multipliers
        modified_returns = returns.copy()
        for i, asset in enumerate(returns.columns):
            # Apply multiplier to the mean return
            if asset in scenario["multipliers"]:
                multiplier = scenario["multipliers"][asset]
                # For negative multipliers, we actually want to flip the sign of returns
                if multiplier < 0:
                    modified_returns[asset] = -returns[asset] * abs(multiplier)
                else:
                    modified_returns[asset] = returns[asset] * multiplier

        # Calculate performance of each portfolio under this scenario
        results = []
        for port_name, weights in portfolios.items():
            expected_return, volatility, sharpe = calculate_portfolio_performance(
                weights, modified_returns
            )
            results.append(
                {
                    "Portfolio": port_name,
                    "Expected Return": expected_return,
                    "Volatility": volatility,
                    "Sharpe Ratio": sharpe,
                }
            )

        # Create DataFrame
        results_df = pd.DataFrame(results)
        results_df = results_df.sort_values("Expected Return", ascending=False)

        # Format for display
        display_df = results_df.copy()
        display_df["Expected Return"] = display_df["Expected Return"].apply(lambda x: f"{x:.2%}")
        display_df["Volatility"] = display_df["Volatility"].apply(lambda x: f"{x:.2%}")
        display_df["Sharpe Ratio"] = display_df["Sharpe Ratio"].apply(lambda x: f"{x:.2f}")

        return display_df, results_df

    # Handle dropdown change
    def on_scenario_change(change):
        scenario_name = change["new"]
        with scenario_output:
            clear_output()

            # Display scenario description
            scenario = scenarios[scenario_name]
            display(HTML(f"<h3>{scenario_name} Scenario</h3>"))
            display(HTML(f"<p><i>{scenario['description']}</i></p>"))

            # Display asset return multipliers
            multipliers = pd.DataFrame(
                list(scenario["multipliers"].items()), columns=["Asset", "Return Multiplier"]
            )
            display(multipliers)

            # Analyze and display results
            display_df, results_df = analyze_scenario(scenario_name)
            display(HTML("<h4>Portfolio Performance Under This Scenario</h4>"))
            display(display_df.style.background_gradient(subset=["Expected Return"], cmap="RdYlGn"))

            # Plot returns comparison
            plt.figure(figsize=(10, 6))
            sns.barplot(x="Portfolio", y="Expected Return", data=results_df, palette="viridis")
            plt.title(f"Expected Returns by Portfolio Under {scenario_name} Scenario")
            plt.ylabel("Expected Annual Return")
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()

            # Display asset allocations of the best performing portfolio
            best_portfolio = display_df.iloc[0]["Portfolio"]
            best_weights = portfolios[best_portfolio]

            display(HTML(f"<h4>Best Performing Portfolio: {best_portfolio}</h4>"))
            display_portfolio(best_weights, returns, f"{best_portfolio} Portfolio Allocation")

    # Connect dropdown change handler
    scenario_dropdown.observe(on_scenario_change, names="value")

    # Display widgets
    display(HTML("<h2>Market Scenario Analysis</h2>"))
    display(HTML("<p>Select a market scenario to see how different portfolios would perform.</p>"))
    display(scenario_dropdown)
    display(scenario_output)

    # Trigger initial display
    on_scenario_change({"new": scenario_dropdown.value})

## Key Takeaways

1. **Diversification matters**: Combining uncorrelated assets can reduce risk without necessarily reducing returns.

2. **The efficient frontier** represents the set of optimal portfolios that provide the highest return for a given level of risk.

3. **Asset allocation** is often more important than individual security selection in determining portfolio performance.

4. **Different market scenarios** affect asset classes differently - no single portfolio is optimal in all market conditions.

5. **Risk and return tradeoffs** are fundamental to investment decisions - higher expected returns generally come with higher risk.

6. **Portfolio optimization** helps identify the best asset mix based on your risk tolerance and return objectives.

7. **Regular rebalancing** helps maintain your target asset allocation and can potentially improve returns by automatically "buying low and selling high."

## Further Reading

- [Modern Portfolio Theory](https://www.investopedia.com/terms/m/modernportfoliotheory.asp) (Investopedia)
- [Efficient Frontier](https://www.investopedia.com/terms/e/efficientfrontier.asp) (Investopedia)
- [Asset Allocation](https://www.investor.gov/introduction-investing/basics/investment-strategies/asset-allocation) (Investor.gov)
- [Portfolio Rebalancing](https://www.investopedia.com/terms/r/rebalancing.asp) (Investopedia)
